# Cuaderno para agregar información de contactabilidad de cliente para iniciativa contagio

In [1]:
import pandas as pd
import pyodbc as odbc
from datetime import datetime

In [2]:
# Leer los datos de los archivos Excel
p1 = pd.read_excel("../db/contagio_p1.xlsx", sheet_name="Consolidado")
p2 = pd.read_excel("../db/contagio_p2.xlsx", sheet_name="Ofertas")

# Concatenar ambos DataFrames
final = pd.concat([p1, p2])

# Crear una nueva columna con el conteo de columnas relevantes no nulas
final['INFO_SCORE'] = final[["NO_PRECALIFICACION", "OBSERVACIONES"]].notnull().sum(axis=1)

# Ordenar el DataFrame basado en 'INFO_SCORE' de mayor a menor
final = final.sort_values(by='INFO_SCORE', ascending=False)

# Eliminar duplicados manteniendo el registro con mayor información
final = final.drop_duplicates(
    subset=["CODIGO_CLIENTE", "NOMBRE", "BANCO", "CAPITAL_ORIGINAL", "SALDO", "ESTADO", "SALARIO_LIQUIDO"], 
    keep="first"
)

# Eliminar la columna auxiliar 'INFO_SCORE'
final = final.drop(columns=['INFO_SCORE'])

# Guardar el resultado en un archivo Excel
final.to_excel("../db/Salidas/precas_finales.xlsx", index=False)

## Leer datos del CP

In [3]:
df = pd.read_excel("../db/Salidas/precas_finales.xlsx")
mascara_falta_precalificar = (df["OBSERVACIONES"].isna()) & (df["NO_PRECALIFICACION"].isna()) & (df["RCI_INTERNO_FINAL"].isna()) & (df["RCI_GLOBAL_FINAL"].isna())
faltan_precalificar = df.loc[mascara_falta_precalificar]
df = df.loc[~mascara_falta_precalificar]
len(df)

1160

In [4]:
# Rellenar vacíos con monto final
mascara_monto_final = df["MONTO_FINAL"].isna()
df.loc[mascara_monto_final, "MONTO_FINAL"] = df.loc[mascara_monto_final, "MONTO_SOLICITADO"]
 
# Cambiar plazo final
mascara_plazo = (df["PLAZO_FINAL"].isna()) | (df["PLAZO_FINAL"] <= 0)
df.loc[mascara_plazo, "PLAZO_FINAL"] = df.loc[mascara_plazo, "PLAZO"]

# Evaluar RCI interno final
mascara_rci = (df["RCI_INTERNO_FINAL"] <= 0) | (df["RCI_INTERNO_FINAL"].isna())
registros_a_evaluar = df.loc[mascara_rci]

mascara_rci_global = (df["RCI_GLOBAL_FINAL"] <= 0) | (df["RCI_GLOBAL_FINAL"].isna())
registros_a_evaluar = pd.concat([registros_a_evaluar, df.loc[mascara_rci_global]])

mascara_cuota = (df["CUOTA_CON_SEGURO"] <= 0) | (df["CUOTA_CON_SEGURO"].isna())
registros_a_evaluar = pd.concat([registros_a_evaluar, df.loc[mascara_cuota]])

mascara_preca = df["NO_PRECALIFICACION"].isna()
registros_a_evaluar = pd.concat([registros_a_evaluar, df.loc[mascara_preca]])
registros_a_evaluar = registros_a_evaluar[registros_a_evaluar["OBSERVACIONES"].isna()]

# Eliminar duplicados de registros a evaluar para envair un registro por cliente
registros_a_evaluar.drop_duplicates(subset=["DPI", "BANCO", "CAPITAL_ORIGINAL", "SALDO", "CATEGORIA"], inplace=True)


In [5]:
registros_a_evaluar.head(3)

,CODIGO_CLIENTE,DPI,NOMBRE,PATRONO,BANCO,CAPITAL_ORIGINAL,SALDO,ESTADO,CATEGORIA,CODIGOEMPLEADO,...,NO PRECALIFICACIÓN,NOMBRE_BANCO,CN,Nombre CN,Departamento,Municipio,Ejecutivo Asignado,CP,OBSERVACIONES,NO_PRECALIFICACION
922,2063570490,2342328621101,Diaz Bonilla Ricardo Enrique,Ministerio Salud Publica Presupuesto,"BANCO AZTECA DE GUATEMALA, S. A.",162.0,146.39,Vencido en Cobro Administrativo,D,990046966,...,41980989,"BANCO AZTECA DE GUATEMALA, S. A.",1621.0,1621-RETALHULEU Delegacion de Creditos,Retalhuleu,Retalhuleu,50908,TORRE 2,NaN,NaN
923,2063570490,2342328621101,Diaz Bonilla Ricardo Enrique,Ministerio Salud Publica Presupuesto,"BANCO AZTECA DE GUATEMALA, S. A.",100.0,91.00,Vencido en Cobro Administrativo,D,990046966,...,41980989,"BANCO AZTECA DE GUATEMALA, S. A.",1621.0,1621-RETALHULEU Delegacion de Creditos,Retalhuleu,Retalhuleu,50908,TORRE 2,NaN,NaN
924,101137687,1812095190713,Chumil Culan Luis Edwin,Ministerio Salud Publica Presupuesto,"BANCO AZTECA DE GUATEMALA, S. A.",7000.0,3586.45,Vencido en Cobro Administrativo,E,9901238989,...,41962553,"BANCO AZTECA DE GUATEMALA, S. A.",2021.0,2021-SANTIAGO ATITLAN Delegacion de Creditos,Solola,Santiago Atitlan,A202,MAZATE,NaN,NaN


Hasta este punto, hemos creado un df, que se llama "registros_a_evaluar", que es el archvio que debería de volver a enviarse a los centros de procesamiento para que corrijan la información. 

In [6]:
# Redefir el df para quedarse solo con los que sí tienen la evaluación del CP válidas
df = df.loc[~mascara_rci & ~mascara_rci_global & ~mascara_cuota & ~mascara_preca]

En este último trozo de código, hemos redefinido el df como un df que solamente contiene registros válidos del CP. Lo que corresponde hacer con estos registros es verificar si las ofertas son válidas o no y discriminar en aquellos que son candidatos a un recrédito no pueden ser parte de la iniciativa.

## Discriminar para recalibrar oferta

In [7]:
# Eliminar registros que no pasan por RCI
"""Estos son los clientes a los que hay que mejorarles la oferta porque no pasan por RCI. Se puede probar solo aumentar
el plazo o pasarlos a recrédito."""
mascara_mejorar_oferta = (df["RCI_INTERNO_FINAL"] > 60)| ((df["RCI_INTERNO_FINAL"] > 0.6) & (df["RCI_INTERNO_FINAL"] <= 3))
mejorar_oferta = df.loc[mascara_mejorar_oferta]


# Evaluar registros que deben ser eliminados: la razón es CCR interna 
mascara_eliminar_iniciativa = df['OBSERVACIONES'].str.contains(r'(?i)\b(ccr int|ccr in)\b', na=False)
eliminar_iniciativa = df.loc[mascara_eliminar_iniciativa]


# Redefinir el df para eliminar lso registros no viables y los de cambio de oferta
df = df.loc[~mascara_mejorar_oferta & ~mascara_eliminar_iniciativa]

C:\Users\mariajose_chinchilla\AppData\Local\Temp\ipykernel_11192\3182271094.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mascara_eliminar_iniciativa = df['OBSERVACIONES'].str.contains(r'(?i)\b(ccr int|ccr in)\b', na=False)


Hasta este punto tenemos varios df definidos. Por un lado, está "registros_a_evaluar", que son los registros que no se ingresaron bien, "faltan_precalificar" es el conjunto de datos que aún no ha sido precalificado; "mejorar_oferta" son los que no pasaron por RCI, "eliminar_iniciativa" son los que no pueden ser parte de la iniciativa porque tienen una CCR interna.

## Buscar información de contactabilidad

Buscamos información de contactabilidad de los clientes que quedan en la base que se trasladará, por lo que almacenamos esos códigos de cliente en una lista que nos permita buscarlos en las bases.

In [8]:
clientes_dpi = tuple(str(dpi) for dpi in df["DPI"].unique().tolist())

servidor = "BTBICC2-VM"
base = "BTMM"
sql_conn = odbc.connect('Driver={SQL Server};'
                      f'Server={servidor};'
                      f'Database={base};'
                      'Trusted_Connection=yes;')

query = f"""SELECT [Codigo Cliente]
      ,[Nombre]
      ,[DPI]
      ,[Email]
      ,[Telefono de Casa]
      ,[Telefono de Trabajo]
      ,[Telefono Movil]
      ,[Otro Telefono]
      ,[Direccion de Casa]
      ,[Direccion de Trabajo]
  FROM [BTMM].[Virtual].[DimCliente]
  WHERE [DPI] IN {clientes_dpi}"""

contactabilidad = pd.read_sql(query, sql_conn)

C:\Users\mariajose_chinchilla\AppData\Local\Temp\ipykernel_11192\1789057466.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  contactabilidad = pd.read_sql(query, sql_conn)


In [9]:
# Ahora pegamos la información del ABF asignado
clientes_cod = tuple(str(cod) for cod in df["CODIGO_CLIENTE"].unique().tolist())
servidor_abf = "BTBICC2-VM"
base_abf = "CARTERA_ASIGNACION"
sql_conn_abf = odbc.connect('Driver={SQL Server};'
                      f'Server={servidor};'
                      f'Database={base_abf};'
                      'Trusted_Connection=yes;')
query_abf = f"""


WITH BASE_PRINCIPAL AS
						(
						
								select distinct

								CODIGO_CLIENTE
								,CODIGO_EJECUTIVO_ASIGNACION
								
								from CARTERA_ASIGNACION.Crecimiento.FACT_CRECIMIENTO
								
								where
								FEC_SALDO = convert(date,DATEADD(day,-1,getdate()),103) 
								and
								CODIGO_CLIENTE in {clientes_cod}
								

						
						),

COLABORADORES_Y_CN	AS	
						
						(

								SELECT
								
								ANTIGUEDAD
								,NOMBRE
								,PUESTO
								,PUESTO_RESUMIDO
								,CASE
										WHEN CONVERT(DATE,FECHA_INFORMACION,103) = CONVERT(DATE,DATEADD(DAY,-1,GETDATE()),103) THEN 'VIGENTE' ELSE 'NO VIGENTE'
								 END AS 'ESTATUS'
								,AGENCIA
								,BANCA
								FROM 
									(
										SELECT DISTINCT
											PKParticipante														AS	ANTIGUEDAD
											,CONCAT(nombre1,' ',nombre2)										AS	NOMBRE
											,PKPuesto															AS	PUESTO
											,  CASE
												    WHEN PKPuesto IN (
												        'ABF Departamental Nororiente', 
												        'ABF Departamental Suroccidente', 
												        'ABF Descentralizada', 
												        'ABF Preferente'
												    ) THEN 'Asesor de bienestar financiero PED'
												    WHEN PKPuesto = 'Ejecutivo' THEN 'Asesor de bienestar financiero BT'
												    ELSE PKPuesto
												END																AS	PUESTO_RESUMIDO
											,PKBANCA															AS	BANCA
											,FECHA																AS	FECHA_INFORMACION
											,PKAgencia															AS	AGENCIA
											,ROW_NUMBER() OVER(PARTITION BY PKParticipante ORDER BY FECHA DESC) AS	FILA
										FROM PGC..PGCParticipante									
									) AS UNICO
								WHERE UNICO.FILA = 1

						),


CENTRO_DE_NEGOCIO	AS
						(
							
							SELECT
							CODIGO_CENTRO_NEGOCIO
							,CENTRO_NEGOCIO
							,REGION
							,SUB_REGION
							,ESTADO
							,ROW_NUMBER() OVER(PARTITION BY CODIGO_CENTRO_NEGOCIO ORDER BY CODIGO_DEPENDENCIA DESC) AS PRIORIDAD
							FROM
							CARTERA_ASIGNACION.Crecimiento.DIM_CENTRO_NEGOCIO
						
						),



PRIMER_COMPARATIVO	AS	
						(
							SELECT 
							
							BASE_PRINCIPAL.CODIGO_CLIENTE
							,BASE_PRINCIPAL.CODIGO_EJECUTIVO_ASIGNACION
							,COLABORADORES_Y_CN.ANTIGUEDAD
							,COLABORADORES_Y_CN.NOMBRE
							,COLABORADORES_Y_CN.PUESTO
							,COLABORADORES_Y_CN.PUESTO_RESUMIDO
							,COLABORADORES_Y_CN.ESTATUS
							,COLABORADORES_Y_CN.BANCA
							,CENTRO_DE_NEGOCIO.CODIGO_CENTRO_NEGOCIO
							,CENTRO_DE_NEGOCIO.CENTRO_NEGOCIO
							,CENTRO_DE_NEGOCIO.REGION
							,CENTRO_DE_NEGOCIO.SUB_REGION
							,CENTRO_DE_NEGOCIO.ESTADO

							FROM BASE_PRINCIPAL
							LEFT JOIN COLABORADORES_Y_CN ON BASE_PRINCIPAL.CODIGO_EJECUTIVO_ASIGNACION = COLABORADORES_Y_CN.ANTIGUEDAD
							LEFT JOIN CENTRO_DE_NEGOCIO ON COLABORADORES_Y_CN.AGENCIA = CENTRO_DE_NEGOCIO.CODIGO_CENTRO_NEGOCIO AND CENTRO_DE_NEGOCIO.PRIORIDAD = 1
						),


SEGUNDO_COMPARATIVO	AS	
						(
							SELECT
							PRIMER_COMPARATIVO.CODIGO_CLIENTE
							,PRIMER_COMPARATIVO.CODIGO_EJECUTIVO_ASIGNACION
							,'SIN EJECUTIVO'	AS		'ANTIGUEDAD'
							,'SIN EJECUTIVO'	AS		'NOMBRE'
							,'SIN EJECUTIVO'	AS		'PUESTO'
							,'SIN EJECUTIVO'	AS		'PUESTO_RESUMIDO'
							,'SIN EJECUTIVO'	AS		'ESTATUS'
							,'SIN EJECUTIVO'	AS		'BANCA'
							,CENTRO_DE_NEGOCIO.CODIGO_CENTRO_NEGOCIO
							,CENTRO_DE_NEGOCIO.CENTRO_NEGOCIO
							,CENTRO_DE_NEGOCIO.REGION
							,CENTRO_DE_NEGOCIO.SUB_REGION
							,CENTRO_DE_NEGOCIO.ESTADO
							FROM PRIMER_COMPARATIVO
							LEFT JOIN CENTRO_DE_NEGOCIO ON RIGHT(PRIMER_COMPARATIVO.CODIGO_EJECUTIVO_ASIGNACION,3) = CENTRO_DE_NEGOCIO.CODIGO_CENTRO_NEGOCIO  AND CENTRO_DE_NEGOCIO.PRIORIDAD = 1
							WHERE
							PRIMER_COMPARATIVO.ANTIGUEDAD IS NULL

						
						
						)


SELECT * FROM PRIMER_COMPARATIVO WHERE NOMBRE IS NOT NULL
UNION ALL
SELECT * FROM SEGUNDO_COMPARATIVO



"""

abf_asignado = pd.read_sql(query_abf, sql_conn_abf)

C:\Users\mariajose_chinchilla\AppData\Local\Temp\ipykernel_11192\3908096742.py:148: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  abf_asignado = pd.read_sql(query_abf, sql_conn_abf)


Ahora pegamos esta información al df para poder trasladarlo al área comercial.

In [10]:
df_temp = df.copy() 
abf_asignado["CODIGO_CLIENTE"] =abf_asignado["CODIGO_CLIENTE"].astype("int64")
df = pd.merge(df, abf_asignado, on="CODIGO_CLIENTE", how="inner")

# Verificar que se encuentre la información de todos los clientes
mascara_no_abf = ~df_temp["DPI"].isin(df["DPI"].unique().tolist())
no_abf = df_temp.loc[mascara_no_abf]
df = df.loc[df["DPI"].isin(df_temp["DPI"].unique().tolist())]
# Almacenar los valores que fueron excluidos en no_abf_asignado
no_abf_asignado = df_temp.loc[mascara_no_abf]

In [11]:
# Ahora pegar contactabilidad
contactabilidad["Codigo Cliente"] = contactabilidad["Codigo Cliente"].astype("int64")
df = pd.merge(df, contactabilidad, how="inner", left_on="CODIGO_CLIENTE", right_on="Codigo Cliente")

# Verificarf que nadie falte de contactabilidad
mascara_no_contactabilidad = ~df_temp["DPI"].isin(df["DPI_x"].unique().tolist())
no_contactabilidad = df_temp.loc[mascara_no_abf]    
df = df.loc[df["DPI_x"].isin(df_temp["DPI"].unique().tolist())]    

# Notemos que se redefinió df para que solo tenga los registros que sí tienen ABF y contactabilidad.

# Exportar todos los datos para compartirlo a las áreas correspondientes.
df.rename(columns={"CODIGO_EJECUTIVO_ASIGNACION": "ABF ASIGNADO",
                   "DPI_x": "DPI",
                   "NOMBRE_y": "ABF_NOMBRE", "NOMBRE_x": "NOMBRE_CLIENTE",
                   "TASA2": "TASA", "NOMBRE_BANCO": "NOMBRE_BANCO_DEUDA_A_COMPRAR"}, inplace=True)

cols_exportar = ["NO_PRECALIFICACION", "ABF ASIGNADO", "ABF_NOMBRE", "BANCA",
                 "CODIGO_CENTRO_NEGOCIO", "CENTRO_NEGOCIO", "REGION",
                 "SUB_REGION", "CODIGO_CLIENTE", "DPI", "Telefono Movil",
                 "Telefono de Trabajo", "Telefono de Casa",
                 "Direccion de Casa", "Direccion de Trabajo",
                 "MONTO_SOLICITADO", "TASA", "PLAZO",
                 'RCI_INTERNO_FINAL', 'RCI_GLOBAL_FINAL',
                 "CUOTA_CON_SEGURO", "NOMBRE_BANCO_DEUDA_A_COMPRAR", "CP"]


In [12]:
fecha = str(datetime.today().day) + "-" + str(datetime.today().month) + "-" + str(2025) 
df[cols_exportar].to_excel(f"../db/Salidas/Detalle_ofertas_{fecha}.xlsx")

# Exportar el resto según área que tenga que recibir
# Exportar los que no tienen contactabilidad o ABF
no_abf_asignado["OBSERVACION"] = "No se encontró ABF asignado"
no_contactabilidad["OBSERVACION"] = "No se encontró contactabilidad del cliente"
revision_contactabilidad_abf = pd.concat([no_abf, no_contactabilidad])
revision_contactabilidad_abf.to_excel(f"../db/Salidas/No_contactabilidad_{fecha}.xlsx")

# Ahora regresar a CP aquellos que hayan incumplido alguna regla de las definidas anteriormente
faltan_precalificar["MOTIVO DE DEVOLUCIÓN"] = "No se ha precalificado al cliente"
registros_a_evaluar["MOTIVO DE DEVOLUCIÓN"] = "No se ingresó algún dato necesario: revisar No. precalificación, cuota, rci, plazo o monto."
regresar_cp = pd.concat([faltan_precalificar, registros_a_evaluar])
regresar_cp.to_excel(f"../db/Salidas/Regreso_a_CP_{fecha}.xlsx")

# Guardar los que tienen que ser revisados por Emilene para mejorar la oferta
mejorar_oferta.to_excel(f"../db/Salidas/Mejorar_oferta_{fecha}.xlsx")

## Ver boosters

In [2]:
viables = pd.read_excel("../db/Salidas/Detalle_ofertas_7-1-2025.xlsx")
viables.head(3)

,NO_PRECALIFICACION,ABF ASIGNADO,ABF_NOMBRE,BANCA,CODIGO_CENTRO_NEGOCIO,CENTRO_NEGOCIO,REGION,SUB_REGION,CODIGO_CLIENTE,DPI,...,Direccion de Trabajo,MONTO_SOLICITADO,TASA,TASA.1,PLAZO,RCI_INTERNO_FINAL,RCI_GLOBAL_FINAL,CUOTA_CON_SEGURO,NOMBRE_BANCO_DEUDA_A_COMPRAR,CP
0,42119193,52621,Christian Diaz,Canales,260.0,PLAZA GUADALUPE,NOR_ORIENTE,NOR_ORIENTE_2,372473,3858233551901,...,NaN,19300,13.0,13.0,84,5.80,55.86,370.30,"BANCO AZTECA DE GUATEMALA, S. A.",ZACAPA
1,42119193,52621,Christian Diaz,Canales,260.0,PLAZA GUADALUPE,NOR_ORIENTE,NOR_ORIENTE_2,372473,3858233551901,...,NaN,19300,13.0,13.0,84,50.80,55.86,370.30,"BANCO DE DESARROLLO RURAL, S. A.",ZACAPA
2,42048748,56384,Paola Orozco,Canales,156.0,SAN PEDRO SACATEPEQUEZ,SUR_OCCIDENTE,SUR_OCCIDENTE_1,100796890,1999357690404,...,21 Avenida Colonia Cipresales,15000,12.0,12.0,144,44.27,64.87,174.82,"BANCO AZTECA DE GUATEMALA, S. A.",PETEN


In [4]:
precas = pd.read_excel("../db/Salidas/precas_finales.xlsx")
precas.head(3)

,CODIGO_CLIENTE,DPI,NOMBRE,PATRONO,BANCO,CAPITAL_ORIGINAL,SALDO,ESTADO,CATEGORIA,CODIGOEMPLEADO,...,NO PRECALIFICACIÓN,NOMBRE_BANCO,CN,Nombre CN,Departamento,Municipio,Ejecutivo Asignado,CP,OBSERVACIONES,NO_PRECALIFICACION
0,372473,3858233551901,Fuentes Estrada Gilberto,Estado (pasivas),"BANCO AZTECA DE GUATEMALA, S. A.",7550.0,3291.03,Vigente,C,263656,...,NaN,"BANCO AZTECA DE GUATEMALA, S. A.",NaN,NaN,NaN,NaN,NaN,ZACAPA,Cliente con CCR mayor a 6 meses no ducumentada.,42119193
1,100796890,1999357690404,Pichiya Pichiya Timoteo,Ministerio De Gobernacion,"BANCO AZTECA DE GUATEMALA, S. A.",625.0,437.50,Vencido en Cobro Administrativo,C,9901207931,...,NaN,"BANCO AZTECA DE GUATEMALA, S. A.",NaN,NaN,NaN,NaN,NaN,PETEN,Cliente debe actualizar datos,42048748
2,100686497,2083481900401,Saquil Siquinajay De Suyuc Maritza Liliana,Ministerio De Educacion,"BANCO AZTECA DE GUATEMALA, S. A.",1045.0,647.90,Vencido en Cobro Administrativo,C,9901201680,...,NaN,"BANCO AZTECA DE GUATEMALA, S. A.",NaN,NaN,NaN,NaN,NaN,PETEN,Cliente canceló deuda con Credito No. 01241220...,DEUDA CANCELADA


In [6]:
viables = pd.read_excel("../db/Salidas/Detalle_ofertas_7-1-2025.xlsx")
clientes_viables = viables["CODIGO_CLIENTE"].unique().tolist()
precas = precas[precas["CODIGO_CLIENTE"].isin(clientes_viables)]

In [15]:
precas.to_excel("../db/Detalle_viables_contagio.xlsx")